In [62]:
import pandas as pd
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from dotenv import load_dotenv
import os
from datetime import datetime

load_dotenv()
uri = os.getenv('DATABASE_URL')

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [52]:
db = client["aus_prop"]

In [56]:
dates = [x for x in os.listdir("data_inv") if "location" not in x]
dfs = [x for x in os.listdir(f"data_inv/{dates[0]}")]

In [83]:
dfs

['df_data_regions.csv',
 'df_data_states.csv',
 'df_data_suburbs.csv',
 'df_tables_houses.csv',
 'df_tables_town_houses.csv',
 'df_tables_units.csv']

In [98]:
df03 = pd.read_csv(os.path.join("data_inv", "202503", "df_data_suburbs.csv"))
df04 = pd.read_csv(os.path.join("data_inv", "202504", "df_data_suburbs.csv"))

In [102]:
date

'202504'

In [104]:
for name in ["df_data_regions.csv", "df_data_states.csv", "df_data_suburbs.csv"]:
    df = pd.read_csv(os.path.join("data_inv", date, name))
    df["Date"] = datetime.strptime(date, "%Y%m")
    df = df[["Date"] + [x for x in df.columns if "Date" not in x]]

    collection = db[name.split("_", 1)[-1].split(".")[0]]
    collection.insert_many(df.to_dict("index").values())

In [97]:
for name in ["data_regions", "data_states", "data_suburbs"]:
    collection = db[name]
    query = {"Date": datetime.strptime("202504", "%Y%m")}
    result = collection.delete_many(query)

In [39]:
collection = db["locations"]

In [43]:
data = df.to_dict("index")